In [219]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random_deletion

### Загрузим данные

In [247]:
creditg = pd.read_csv("dataset/credit-g.csv", header=True)
krkp = pd.read_csv("dataset/kr-vs-kp.csv", header=True)
segment = pd.read_csv("dataset/segment.csv", header=True)

#### KRKP

In [248]:
krkp.columns = ["col" + str(i) for i in range(1, 38)]

In [249]:
for i in range(1, 38):
    print(np.unique(krkp["col" + str(i)]))

['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['g' 'l']
['f' 't']
['b' 'n' 'w']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['f' 't']
['n' 't']
['nowin' 'won']


In [250]:
for i in range(1, 38):
    krkp["col" + str(i)] = LabelEncoder().fit_transform(krkp["col" + str(i)])

In [251]:
for i in range(1, 38):
    print(np.unique(krkp["col" + str(i)]))

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


In [252]:
krkp_y = krkp["col37"]
krkp.drop("col37", axis=1, inplace=True)

In [253]:
krkp = np.array(krkp, dtype='float')
krkp_y = np.array(krkp_y)

In [254]:
krkp

array([[ 0.,  0.,  0., ...,  1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1.,  1.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  1.,  0.,  0.],
       [ 1.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  1., ...,  0.,  0.,  0.]])

##### 5 % пропущенных значений во всех объектах и всех признаках.

In [255]:
krkp_del = random_deletion.make_missing_value(krkp, del_fraction=0.05)

In [256]:
krkp_del

array([[  0.,   0.,   0., ...,  nan,   1.,   0.],
       [  0.,   0.,   0., ...,   1.,   1.,   0.],
       [  0.,   0.,   0., ...,   1.,   1.,   0.],
       ..., 
       [  1.,   0.,   0., ...,   1.,   0.,   0.],
       [  1.,   0.,   1., ...,   0.,   0.,   0.],
       [  1.,   0.,  nan, ...,   0.,   0.,   0.]])

In [257]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

In [258]:
cv = KFold(krkp_y.shape[0], n_folds=10, shuffle=True, random_state=798)
clf = RandomForestClassifier(n_jobs=-1, random_state=567)

##### 1. Без пропущенных значений.

In [259]:
print(np.mean(cross_val_score(clf, krkp, krkp_y, scoring='accuracy', cv=cv)))

0.989672805643


##### 2. Игнорируем объекты с пропущенными значениями.

In [260]:
mask = np.sum(krkp_del != krkp_del, axis=1) == 0
krkp_del_ignore = krkp_del[mask]
krkp_y_ignore = krkp_y[mask]
print('count =', mask.sum())

count = 522


In [261]:
print(np.mean(cross_val_score(clf, krkp_del_ignore, krkp_y_ignore, scoring='accuracy', cv=10)))

0.900535729531


##### 3. Заполнение специальным значение (-1).

In [262]:
mask = krkp_del != krkp_del
krkp_del_special = np.array(krkp_del)
krkp_del_special[mask] = -1

In [264]:
print(np.mean(cross_val_score(clf, krkp_del_special, krkp_y, scoring='accuracy', cv=cv)))

0.959317202194


##### 4. Заполнение самым частым значением.

In [271]:
from scipy.stats.mstats import mode

krkp_del_common = np.array(krkp_del)
for i in range(krkp_del_common.shape[1]):
    mask = krkp_del_common[:, i] != krkp_del_common[:, i]
    krkp_del_common[mask, i] = mode(krkp_del_common[np.logical_not(mask), i])[0][0]

In [266]:
print(np.mean(cross_val_score(clf, krkp_del_common, krkp_y, scoring='accuracy', cv=cv)))

0.95024588558


##### 5. Заполнение с помощью SVD.

In [280]:
import data_recovery
from importlib import reload
reload(data_recovery)

krkp_del_svd = data_recovery.svd_imputer(krkp_del, rank=26)

In [281]:
print(np.mean(cross_val_score(clf, krkp_del_svd, krkp_y, scoring='accuracy', cv=cv)))

0.963698079937


In [ ]:
accuracy = []
for i in range(1, 37):
    krkp_del_svd = data_recovery.svd_imputer(krkp_del, rank=i)
    accuracy.append(np.mean(cross_val_score(clf, krkp_del_svd, krkp_y, scoring='accuracy', cv=cv)))
plt.plot(list(range(1, 37)), accuracy)